In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import random
import plotly.express as px
import time
import seaborn as sns
sns.set_theme('talk', palette=sns.color_palette('Set2'))
pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 150

In [2]:
infile = open('..//3_k-means_clustering//apartment_data_frame_clustered','rb')
frame = pkl.load(infile)
infile.close()

In [3]:
frame.head()

,city,district,voivodeship,localization_y,localization_x,market,offer_type,area,rooms,floor,floors,build_yr,price,url,price_of_sqm,cluster
0,Gdańsk,Chełm,pomorskie,54.33990,18.619210,aftermarket,sprzedaz,69.00,3,4,4,2019.0,1099000.0,https://gratka.pl/nieruchomosci/mieszkanie-gdansk-chelm-ul-suchanka/ob/32020549,15927.536232,18
1,Konin,NaN,wielkopolskie,52.31940,18.294290,aftermarket,sprzedaz,41.00,2,2,4,2022.0,410000.0,https://gratka.pl/nieruchomosci/mieszkanie-konin/ob/32410147,10000.000000,548
2,Tczew,tczewski,pomorskie,54.09408,18.791000,aftermarket,sprzedaz,30.52,1,1,3,1900.0,209000.0,https://gratka.pl/nieruchomosci/mieszkanie-tczew-ul-mostowa/ob/32126131,6847.968545,47
3,Kraków,Łobzów,małopolskie,50.07573,19.929994,aftermarket,sprzedaz,52.00,3,0,4,1980.0,767000.0,https://gratka.pl/nieruchomosci/mieszkanie-krakow-lobzow/ob/32348887,14750.000000,521
4,Kraków,NaN,małopolskie,50.06038,19.949118,aftermarket,sprzedaz,34.46,2,0,4,1907.0,638000.0,https://gratka.pl/nieruchomosci/mieszkanie-krakow-ul-kollataja/ob/31022867,18514.219385,343


In [4]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32412 entries, 0 to 32411
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            32412 non-null  object 
 1   district        27423 non-null  object 
 2   voivodeship     32412 non-null  object 
 3   localization_y  32412 non-null  float64
 4   localization_x  32412 non-null  float64
 5   market          32412 non-null  object 
 6   offer_type      32412 non-null  object 
 7   area            32412 non-null  float64
 8   rooms           32412 non-null  int32  
 9   floor           32412 non-null  int32  
 10  floors          32412 non-null  int32  
 11  build_yr        32412 non-null  float64
 12  price           32412 non-null  float64
 13  url             32412 non-null  object 
 14  price_of_sqm    32412 non-null  float64
 15  cluster         32412 non-null  int32  
dtypes: float64(6), int32(4), object(6)
memory usage: 3.5+ MB


In [5]:
to_drop = ['city','district','voivodeship','localization_y','localization_x','price_of_sqm','url','offer_type']
frame.drop(to_drop, axis=1, inplace=True)

## creating dummy variables

In [6]:
for column in ['cluster','rooms','floor','floors']:
    frame[column] = frame[column].astype('str')

In [7]:
frame = pd.get_dummies(frame, drop_first=True)

In [8]:
frame

,area,build_yr,price,market_primary_market,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,floor_1,floor_10,floor_11,floor_12,floor_13,floor_14,...,cluster_86,cluster_87,cluster_88,cluster_89,cluster_9,cluster_90,cluster_91,cluster_92,cluster_93,cluster_94,cluster_95,cluster_96,cluster_97,cluster_98,cluster_99
0,69.00,2019.0,1099000.0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,41.00,2022.0,410000.0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30.52,1900.0,209000.0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,52.00,1980.0,767000.0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,34.46,1907.0,638000.0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32407,73.40,1970.0,285000.0,0,0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32408,63.60,2023.0,559680.0,1,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32409,51.00,1975.0,175000.0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32410,79.20,1990.0,430000.0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## train test split

In [9]:
# from sklearn.model_selection import ShuffleSplit

# X = frame.drop('price',axis=1)
# y = frame.price

# ss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

# train_index, test_index = next(ss.split(X))

# X_train = X.loc[train_index,:]
# y_train = y.loc[train_index]

# X_test = X.loc[test_index,:]
# y_test = y.loc[test_index]

In [10]:
from sklearn.model_selection import train_test_split

X = frame.drop('price',axis=1)
y = frame.price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## data save

In [11]:
outfile = open('X_train','wb')
pkl.dump(X_train, outfile)
outfile.close()

outfile = open('X_test','wb')
pkl.dump(X_test, outfile)
outfile.close()

outfile = open('y_train','wb')
pkl.dump(y_train, outfile)
outfile.close()

outfile = open('y_test','wb')
pkl.dump(y_test, outfile)
outfile.close()